In [1]:
import os 
import warnings
import time
import numpy as np
import scipy.stats
import sys
import sklearn
import sklearn.datasets

from pyspark.sql import SparkSession
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("feature_selection") \
    .getOrCreate()

sc=spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 17:16:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/26 17:16:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/26 17:16:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
#train_sessions_engineered = spark.read.csv('../Data/session_engineered_features.csv',header=False,
#                                          inferSchema=True)
train_sessions_engineered = spark.read.load('../Data/session_engineered_features.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true').cache()

train_purchases = spark.read.load('../Data/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

22/05/26 17:16:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
train_purchases.printSchema()

root
 |-- session_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- date: string (nullable = true)



In [4]:
# BINING 
from pyspark.ml.feature import Bucketizer
to_bin = ['session_time','most_time_spent_on_item','mean_time','std_time']#+ [str(i+1) for i in range(25)]
for col in to_bin:
    print(col)
    column_values = train_sessions_engineered.select(col).collect()
    splits_list = [0]
    for i in range(10):
        p = 10 * (i+1)
        splits_list.append(np.percentile(column_values, p))
    splits_list.append(float('Inf'))
    bucketizer = Bucketizer(splits=list(np.unique(splits_list)),inputCol=col, outputCol=col+"_binned")
    train_sessions_engineered = bucketizer.setHandleInvalid("keep").transform(train_sessions_engineered)
    train_sessions_engineered = train_sessions_engineered.drop(col)
    train_sessions_engineered = train_sessions_engineered.withColumnRenamed(col+"_binned",col)

session_time


most_time_spent_on_item
mean_time
std_time


In [5]:
train_sessions_engineered.take(1)

[Row(session_id=28560.0, season=3.0, day_period=2.0, month=8.0, year=0.0, item_most_time_spent=5.0, most_frequently_bought_for_time_spent=28.0, least_frequently_bought_for_time_spent=25.0, item_most_visited=-1.0, number_o_visit=-1.0, number_o_revisited_items=0.0, most_frequently_bought_for_most_revisited=5.0, first_item_visited=23.0, last_item_visited=26.0, normalized_features_vector=5.0, 1=4.0, 2=0.0, 3=0.0, 4=0.0, 5=0.0, 6=0.0, 7=0.0, 8=0.1666666716337204, 9=0.0, 10=0.0, 11=0.0, 12=0.0, 13=0.0, 14=0.0, 15=0.0, 16=0.0, 17=0.0, 18=0.0, 19=0.0, 20=0.8333333134651184, 21=0.0, 22=0.0, 23=0.0, 24=0.0, 25=0.0, _45=0.0, session_time=3.0, most_time_spent_on_item=3.0, mean_time=3.0, std_time=1.0)]

In [6]:
# join the dataframes on the session ids and drop useless columns
train_df = train_sessions_engineered.join(train_purchases,train_sessions_engineered.session_id == train_purchases.session_id,"inner" ).cache()
for col in ['session_id','date']:
    train_df = train_df.drop(col)

In [7]:
train_df.take(1) 

[Row(season=1.0, day_period=2.0, month=3.0, year=1.0, item_most_time_spent=23.0, most_frequently_bought_for_time_spent=11.0, least_frequently_bought_for_time_spent=12.0, item_most_visited=-1.0, number_o_visit=-1.0, number_o_revisited_items=0.0, most_frequently_bought_for_most_revisited=5.0, first_item_visited=23.0, last_item_visited=23.0, normalized_features_vector=23.0, 1=2.0, 2=0.0, 3=0.0, 4=0.0, 5=0.0, 6=0.0, 7=0.0, 8=0.0, 9=0.0, 10=0.0, 11=0.0, 12=0.0, 13=0.0, 14=0.0, 15=0.0, 16=0.0, 17=1.0, 18=0.0, 19=0.0, 20=0.0, 21=0.0, 22=0.0, 23=0.0, 24=0.0, 25=0.0, _45=0.0, session_time=3.0, most_time_spent_on_item=5.0, mean_time=4.0, std_time=2.0, item_id=1640)]

In [8]:
train_df.write.option("header",True).csv('/PROJ/Data/data_processed_for_model.csv')

In [19]:
train_df = spark.read.load('/PROJ/Data/data_processed_for_model.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true').cache()

22/05/26 17:25:45 WARN CacheManager: Asked to cache already cached data.        


# NAIVE BAYES

In [20]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils

def if_p(row):
    row_bis = []
    for count,elem in enumerate(row):
        if elem == -1:
            row_bis.append(0)
        else:
            row_bis.append(elem)
    return row_bis

train_df = train_df.rdd.map(if_p)

def labelData(data):
    return data.map(lambda x: LabeledPoint(int(x[-1]), x[:-1]))

training_data, testing_data = labelData(train_df).randomSplit([0.9, 0.1])

model = NaiveBayes.train(training_data)

In [48]:
import numpy
from pyspark.mllib.linalg import _convert_to_vector
from pyspark import RDD

# define custom predict function 
def custom_predict(model_bayes, x):
    labels_ = model_bayes.labels
    pi = model_bayes.pi
    theta = model_bayes.theta
    if isinstance(x, RDD):
        return x.map(lambda v: custom_predict(model_bayes,v))
    x = _convert_to_vector(x)
    x = pi + x.dot(theta.transpose())
    indices = np.argsort(x)[-100:]
    top100 = labels_[indices]
    return top100 #labels_[numpy.argmax(pi + x.dot(theta.transpose()))]


In [49]:
testing_data.map(lambda p: (model.predict(p.features), p.label)).take(20)

[(12690.0, 26139.0),
 (8290.0, 19821.0),
 (10338.0, 2173.0),
 (24429.0, 16867.0),
 (2927.0, 16627.0),
 (16365.0, 22419.0),
 (10223.0, 9371.0),
 (11208.0, 23510.0),
 (5339.0, 23785.0),
 (3522.0, 19818.0),
 (26977.0, 11681.0),
 (8622.0, 4917.0),
 (13767.0, 6182.0),
 (10338.0, 21499.0),
 (20281.0, 164.0),
 (21043.0, 20037.0),
 (12540.0, 15573.0),
 (158.0, 10997.0),
 (26977.0, 8861.0),
 (6355.0, 27315.0)]

In [50]:
model.predict(testing_data.take(4)[3].features)

24429.0

# TOP 100 with bayes pred

In [51]:
top100 = custom_predict(model,testing_data.take(4)[3].features)
print(top100)

[18581. 10896. 14457.  4981. 18432. 15862. 23019. 16326. 18992.  6237.
 25420. 16462. 23571. 27051. 18031.   897.   527.  5977. 18214. 12606.
 18832. 26977. 18665. 13403.  1043. 20152.  4598.  2458. 25736.   228.
  6089. 14961.  8795. 17806.  4130.  4885. 22821.   375. 27658.  3722.
 25287. 18218.  4131.  3037. 15342.  8585. 21420. 15838.  3273.  8684.
 17375. 27872. 20247. 21257.  5350. 23252. 24803. 11284. 23793. 28101.
  6129. 13683. 28030. 27773.  4940.  9849. 26662. 18594.  4179. 16844.
 11186. 26454. 18375.  7594. 17659. 13914. 23212. 10163. 27822. 12158.
  7977.  5276. 11917. 15067. 13564. 26555.  6818.  5833.  9494. 21587.
 15870. 23700.  6361. 16924.  2158.  1172.  2674. 19310.  8119. 24429.]


In [60]:
results_top100 = testing_data.map(lambda p: (p.label in custom_predict(model,p.features), p.label))

In [61]:
results_top100.take(10)

[(False, 26139.0),
 (False, 19821.0),
 (False, 2173.0),
 (False, 16867.0),
 (True, 16627.0),
 (False, 22419.0),
 (False, 9371.0),
 (False, 23510.0),
 (False, 23785.0),
 (False, 19818.0)]

In [54]:
results_top100.map(lambda x : (x[0],(1))).reduceByKey(lambda x,y : np.add(x,y)).take(2)

[(False, 172319), (True, 28251)]

In [57]:
np.where(np.array([1,2,3,4,5,6,7,8]) == 0 )

(array([], dtype=int64),)

In [82]:
def score(top_100,label):
    if label in top_100:
        return (np.where(np.array(top_100) == label)[0][0] + 1)/100
    else:
        return 0

In [85]:
testing_data.map(lambda p: (score(custom_predict(model,p.features),p.label))).mean()

0.08662377224908996